In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
import glob
import re
import datetime
import seaborn as sns

# Chargement des données

In [2]:
path = 'C:/Users/Anastasiia/Documents/BigData-IA/kc_house_data.csv'
data = pd.read_csv(path, delimiter=',', header=0, index_col=None)
data["date"] = pd.to_datetime(data["date"],format='%Y-%m-%d') #Transformation de la colonne "date"
price = data["price"]
surface = data["sqft_living"]
grade = data["grade"]
ppm = price/surface

# Random forest

In [10]:
#Librairies
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [43]:
#Separation du dataset en X(variables) et Y(cibles)
X = data.drop(['price','date','id'], axis=1)
Y = data['price']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=5) #Split Test/Train 20/80%


In [44]:
#Test de l'algorithme
regressor = RandomForestRegressor(n_estimators= 50, random_state=0)  
regressor.fit(X_train, Y_train)  
Y_pred = regressor.predict(X_test) 

rmse = (np.sqrt(mean_squared_error(Y_test, Y_pred)))
r2 = r2_score(Y_test, Y_pred)

print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print("\n")

RMSE is 126301.08510661259
R2 score is 0.8830771749503906




# Recherche des parametres optimales pour maximiser R2

In [39]:
#On essaye de varier le nombre d'arbres
trees = [20, 40, 50, 70, 90]
score_trees = []
for elt in trees:
    regressor = RandomForestRegressor(n_estimators=elt, random_state=0)  
    regressor.fit(X_train, Y_train)  
    Y_pred = regressor.predict(X_test) 
    r2 = r2_score(Y_test, Y_pred)
    score_trees.append(r2)

In [40]:
score_trees #max à 50

[0.87912405617487321,
 0.8842931566859793,
 0.88514432553533096,
 0.88344031809868206,
 0.88307869774168124]

In [41]:
# #On essaye de varier le paramètre random
random = [0, 5, 10, 15, 20]
score_random = []
for elt in random:
    regressor = RandomForestRegressor(n_estimators=50, random_state=elt)  
    regressor.fit(X_train, Y_train)  
    Y_pred = regressor.predict(X_test) 
    r2 = r2_score(Y_test, Y_pred)
    score_random.append(r2)

In [42]:
score_random # max à 0

[0.88514432553533096,
 0.88167567311518558,
 0.88426301150341313,
 0.88050527263394651,
 0.87957167374056522]

In [47]:
#On essaye de varier la profondeur de chaque arbre
depth = [1, 5, 10, 15, 20]
score_depth = []
for elt in depth:
    regressor = RandomForestRegressor(n_estimators=50, max_depth=elt, random_state=0)  
    regressor.fit(X_train, Y_train)  
    Y_pred = regressor.predict(X_test) 
    r2 = r2_score(Y_test, Y_pred)
    score_depth.append(r2)

In [48]:
score_depth #max à max_depth = None

[0.33380947932828131,
 0.78156675708501,
 0.86757655852940907,
 0.88058623387789214,
 0.88181689972713506]

In [49]:
#Pour l'instant le R2max est de 0.883 